In [9]:
import pandas as pd
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm, trange
storage = "/Volumes/easystore/Drones/"

calls_for_service = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"{storage}/calls-for-service-with-census-data.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)
flight_data = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../data/all-flights-manifest.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

In [16]:
flight_data = flight_data[flight_data['department']=='cvpd']
flight_data['match']=flight_data['incident_id'].astype(str).apply(lambda x: x[-5:])
flight_data['drone-day']=pd.to_datetime(flight_data['date']).dt.date


/var/folders/nq/51nwndf52yqcj9x8h10zjpg80000gn/T/ipykernel_32775/3284868300.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  flight_data['drone-day']=pd.to_datetime(flight_data['date']).dt.date


In [17]:
calls_for_service['match']=calls_for_service['Incident No.'].astype(str).apply(lambda x: x[-5:])
calls_for_service['cfs-day']=pd.to_datetime(calls_for_service['Date']).dt.date

In [20]:
matches = flight_data[flight_data['match'].isin(calls_for_service['match'])]

,lat_map,lon_map,time,id,date,time_s,address_map,incident_id,type,department,match,drone-day
0,32.6224144,-117.0504452,5:34pm,8f0f4842f6dbf20b94ed2df3021b5ee5,4-30-21,5:34pm,Inkopah St/ Monserate Ave,CV2105442,Traffic Collision,cvpd,05442,2021-04-30
2,32.63996724,-117.08270561,12:36pm,63e43c48d029a60db4a6cf8342e8a2d9,4-30-21,12:36pm,300 Block of H St,CVL035519,Lude activity,cvpd,35519,2021-04-30
3,32.7115419,-117.1610577,11:38am,b182a693dcd6a99609d0031ffade5dd7,4-30-21,11:38am,600 Fourth Ave,CVL035512,Domestic violence,cvpd,35512,2021-04-30
5,32.612919,-116.974738,10:43am,b5ccd5e4284adfa02ace3ed0fb89a837,4-30-21,10:43am,1825 Magdalena Ave,CVL035490,Welfare Check,cvpd,35490,2021-04-30
6,32.7115419,-117.1610577,10:37am,7bc7376996192a31414d30bb29ee0229,4-30-21,10:37am,600 Fourth Ave,CVL035493,Suspicious Circumstances,cvpd,35493,2021-04-30
...,...,...,...,...,...,...,...,...,...,...,...,...
10997,32.6137073,-117.0691405,2:48pm,c3c86e9941fa457323fba31583193c77,9-01-23,2:48pm,Third Ave./ Naples St.,CVL75025,Collision,cvpd,75025,2023-09-01
10998,32.6295199,-117.0798809,2:23pm,48322223a2b0290178fe088f0388bab5,9-01-23,2:23pm,400 I St.,CVL75022,Fight,cvpd,75022,2023-09-01
10999,32.627937,-117.0412309,10:36am,ecf83badca5a43ac77228478fae8583a,9-01-23,10:36am,500 Telegraph Canyon Rd.,CVL74960,Drunk Driver,cvpd,74960,2023-09-01
11000,32.639897,-117.094566,10:24am,b4063b9cb4286b49ecda0ac31425b783,9-01-23,10:24am,230 Broadway,CVL74958,Domestic Violence,cvpd,74958,2023-09-01


In [26]:
matches_joined_cfs = pd.merge(matches,calls_for_service,how='left',on='match')
matches_joined_cfs = matches_joined_cfs[['time', 'id', 'date', 'time_s', 'address_map',
       'incident_id', 'type', 'Incident No.', 'Date',
       'Description', 'Priority', 'Disposition', 'Call Source',
       'Block Location', 'Case No', 'Address','drone-day','cfs-day']]

In [28]:
matches_joined_cfs = matches_joined_cfs[matches_joined_cfs['drone-day'] == matches_joined_cfs['cfs-day']]
matches_joined_cfs.head()

,time,id,date,time_s,address_map,incident_id,type,Incident No.,Date,Description,Priority,Disposition,Call Source,Block Location,Case No,Address,drone-day,cfs-day
2038,8:04pm,6e43fee7c42973b78794b68f37d2d63a,7-31-21,8:04pm,Broadway/ H St.,CVL63747,Weapon Threat,210731L00063747,2021-07-31 20:05:13.180,Disturbance - General,2,ASSISTED,citizen,BROADWAY & H ST,NaN,"BROADWAY & H ST, Chula Vista, CA, 91910",2021-07-31,2021-07-31
2041,7:36pm,a9eab4c657adb497824246bacb28a8ef,7-31-21,7:36pm,605 Anita St.,CVL63740,Suspicious Person,210731L00063740,2021-07-31 19:02:42.460,Suspicious Person,3,HANDLED NO REPORT,citizen,500 BLOCK ANITA ST,NaN,"500 BLOCK ANITA ST, Chula Vista, CA, 91911",2021-07-31,2021-07-31
2044,7:28pm,1cc1c25ea3a4a79898d304c670f86ef2,7-31-21,7:28pm,00 Montebello St.,CVL63741,Suspicious Person,210731L00063741,2021-07-31 19:11:18.977,Suspicious Person,3,HANDLED NO REPORT,citizen,30 BLOCK MONTEBELLO ST,NaN,"30 BLOCK MONTEBELLO ST, Chula Vista, CA, 91910",2021-07-31,2021-07-31
2046,6:54pm,2e04de76bd8069328dc75642b5c0ca9c,7-31-21,6:54pm,Bonita Rd./ Plaza Bonita Rd.,CVL63738,Unknown Problem,210731L00063738,2021-07-31 19:00:32.523,Miscellaneous Assistance,2,ASSISTED,citizen,BONITA RD & PLAZA BONITA RD,NaN,"BONITA RD & PLAZA BONITA RD, Chula Vista, CA, ...",2021-07-31,2021-07-31
2049,6:30pm,bfa52c039ee731b085b7ea784728ae0f,7-31-21,6:30pm,NaN,Cvl063686,Barricaded subject.,210731L00063686,2021-07-31 15:05:18.990,[Protected Call Type],3,ASSISTED,citizen,100 BLOCK PALOMAR ST,2109611.0,"100 BLOCK PALOMAR ST, Chula Vista, CA, 91911",2021-07-31,2021-07-31


In [30]:
not_matches = flight_data[~flight_data['match'].isin(calls_for_service['match'])]

In [37]:
flight_data[flight_data['incident_id'].isna()].columns

Index(['lat_map', 'lon_map', 'time', 'id', 'date', 'time_s', 'address_map',
       'incident_id', 'type', 'department', 'match', 'drone-day'],
      dtype='object')

In [52]:
not_matches.sample(3)

,lat_map,lon_map,time,id,date,time_s,address_map,incident_id,type,department,match,drone-day
10697,32.7138553,-117.1587941,6:26pm,78a12fdaa5c4cf58517a89e7c2b7c11b,8-09-23,6:26pm,600 F Street,Cvl67828,Person down possibly injured,cvpd,7828,2023-08-09
3947,32.599303,-117.0912224,10:11am,f979eadfb9975285eb485c1355eaf7fc,2-02-22,10:11am,Bay Blvd./ Pacific Ave.,CVL9193,Stolen Vehicle,cvpd,L9193,2022-02-02
5075,32.6174209,-117.07166201,10:34am,a44a932e0e254a492eca05f5ae709d9b,4-03-22,10:34am,330 Moss St,NaN,NaN,cvpd,nan,2022-04-03


In [49]:
calls_for_service[calls_for_service['cfs-day'].astype(str)=="2022-01-27"].tail(50)[['Incident No.', 'Date', 'Description', 'Disposition',
       'Call Source', 'Block Location', 'Case No', 'Address']]


,Incident No.,Date,Description,Disposition,Call Source,Block Location,Case No,Address
156535,220127L00007396,2022-01-27 10:51:31.870,Disturbance - General,ASSISTED,citizen,500 BLOCK PEPPERWOOD CT,NaN,"500 BLOCK PEPPERWOOD CT, Chula Vista, CA, 91902"
156549,220127L00007369,2022-01-27 09:31:07.810,Police Impound,OFFICER INITIATED TOW,officer,2100 BLOCK STONECREEK PL,2201212.0,"2100 BLOCK STONECREEK PL, Chula Vista, CA, 91913"
156767,220127L00007520,2022-01-27 16:21:45.223,Incident Follow Up,HANDLED NO REPORT,citizen,1300 BLOCK EL LUGAR ST,NaN,"1300 BLOCK EL LUGAR ST, Chula Vista, CA, 91911"
157817,220127L00007379,2022-01-27 09:51:34.957,Vehicle Pursuit,ASSISTED,citizen,E H ST & HILLTOP DR,2201216.0,"E H ST & HILLTOP DR, Chula Vista, CA, 91910"
158162,220127L00007393,2022-01-27 10:42:12.323,Narcotics,ASSISTED,officer,3200 BLOCK CYPRESS AV,2201227.0,"3200 BLOCK CYPRESS AV, Chula Vista, CA, 91950"
158655,220127L00007483,2022-01-27 14:42:44.783,Non-Injury Collision,ASSISTED,citizen,800 BLOCK BUENA VISTA WY,NaN,"800 BLOCK BUENA VISTA WY, Chula Vista, CA, 91910"
159154,220127L00007395,2022-01-27 10:43:59.620,Reckless Driver,HANDLED NO REPORT,citizen,PALOMAR ST & WALNUT AV,NaN,"PALOMAR ST & WALNUT AV, Chula Vista, CA, 91911"
159358,220127L00007478,2022-01-27 14:19:41.167,PR Contact,HANDLED NO REPORT,citizen,2300 BLOCK CALLE SABROSO,NaN,"2300 BLOCK CALLE SABROSO, Chula Vista, CA, 91914"
159710,220127L00007311,2022-01-27 03:53:03.370,Disturbance - General,HANDLED NO REPORT,citizen,1300 BLOCK ORTEGA ST,NaN,"1300 BLOCK ORTEGA ST, Chula Vista, CA, 0"
159978,220127L00007610,2022-01-27 21:37:01.137,Check The Area,HANDLED NO REPORT,citizen,700 BLOCK ESLA DR,NaN,"700 BLOCK ESLA DR, Chula Vista, CA, 91910"
